In [1]:
try:
    import IPython
except:
    %pip install IPython
    import IPython 
from IPython.display import display, IFrame, HTML, Javascript
HTML("""<link rel="stylesheet" type="text/css" href="src/css/notebook.css"/>""")

#  Transforming Collections Data to Linked Art
# Philadelphia Museum of Art

## Input Data
The notebook uses data from the Philadelphia Museum of Art (PMA). 

The data file is
- `CSV` format
- contains records about works by Georgia O'Keefe and John Ruskin.

#### Further Reading
- [data file](./data/pma/input/Ruskin_Okeeffe.csv)

### 2. Parse CSV File

In [2]:
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

file = './data/pma/input/Ruskin_Okeeffe.csv'

mpg = pd.read_csv(file,low_memory=False)
mpg.head()

,Media,Object Number,Department,Classification,Culture,Period,Display Name,Object Name,Title,Dated,...,Unnamed: 16370,Unnamed: 16371,Unnamed: 16372,Unnamed: 16373,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379
0,0,64-1993-1,PDP,Drawings,NaN,NaN,John Ruskin,Drawings,"Study of a River Bank, Beauvais, France",1846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1995-7-1,PDP,Drawings,NaN,NaN,John Ruskin,Drawings,Beanstalk,Date unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,9-1982-1,AA,Paintings,NaN,NaN,Georgia O'Keeffe,Paintings,Mask with Golden Apple,1921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,48-1993-1,AA,Paintings,NaN,NaN,Georgia O'Keeffe,Paintings,African Mask and Apple,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,250-2003-1,AA,Paintings,NaN,NaN,Georgia O'Keeffe,Paintings,"Red Hills with Pedernal, White Clouds",1936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Data File as Python dictionary
The data file is converted to a Python dictionary, and will be used to transform the collection data for the artwork to JSON-LD. The following shows an example of a single artwork represented in the Python dictionary:

In [3]:
import csv

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

In [4]:
try:
    import json
except:
    !pip install json
    import json 



for obj in allObjects:
    print(json.dumps(obj,indent=2))
    break

{
  "Media": "0",
  "Object Number": "64-1993-1",
  "Department": "PDP",
  "Classification": "Drawings",
  "Culture": "",
  "Period": "",
  "Display Name": "John Ruskin",
  "Object Name": "Drawings",
  "Title": "Study of a River Bank, Beauvais, France",
  "Dated": "1846",
  "Medium": "Watercolor",
  "Dimensions": "H: 125mm  W: 175mm",
  "Description": "",
  "Attribution": "John Ruskin",
  "AttributionSort": "Ruskin John",
  "Credit Line": "",
  "": ""
}


In [5]:

mapp =  {
    "id":"Object Number",
    "accession_number":"Object Number",
    "accession_date": "accession_date",
    "classification" : "Classification",
    "title": "Title",
    "alt_title": "",
    "notes": "Description",
    "date_created":"Dated",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"Period",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "Description",
    "creator":"Display Name",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "physical_technique",
    "physical_description": "physical_description",
    "physical_dimensions": "Dimensions",
    "created_provenance": "Attribution" ,
    "credit_line": "Credit Line",
    "collection" : "Department",
    "current_status" : "",
    "homepage": ""
}

# display transposed dataframe of data mapping
display(pd.DataFrame(mapp, index=[0]).T)


,0
id,Object Number
accession_number,Object Number
accession_date,accession_date
classification,Classification
title,Title
alt_title,
notes,Description
date_created,Dated
date_created_earliest,
date_created_latest,


In [6]:
#  baseURI for JSON-LD document
baseURI = "https://philamuseum.org/collection/object/"


def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                   
                    objProp[prop] = [{"id": baseURI +"creatorid/" + obj[mapp["id"]] ,"name": obj[key],"role":"Artist"}]
                else:
                    objProp[prop] = obj[key]
    objProp["homepage"] = ""
    return objProp

In [7]:
try:
    import cromulent
except:
    !pip install cromulent
    import cromulent
    
from cromulent.model import factory
    
from lib import linkedart as la




# list to hold file names for use with jsonld visualisation dropdown
selectOptions = []
selectOptions = [('Please select an artwork', '')]

outputdir = "./data/pma/output/json/all/"

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp)

    id = obj[mapp.get("id")]
    object_uri = baseURI + id
    objLA=None
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    filename = objProp["id"] + ".json"
    selectOptions.append( ( objProp["title"] + " (" + filename + ")" , filename))
    # write to file 
    text_file = open(outputdir + filename, "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

In [9]:
try:
    import ipywidgets
except:
    %pip install ipywidgets
    import ipywidgets

from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML, Image
from IPython.core.display import Javascript        
 
import os

try:
    import json
except:
    %pip install json
    import json 
    
   
def dropdown_eventhandler(change):
    with open('./src/js/visld.js', 'r') as _jscript:
        code = _jscript.read() + "var file = '" + outputdir + change.new + "';var selector = '#vis';visjsonld(file, selector); "
        display(Javascript(code))
selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')

display(selectObject)

Dropdown(options=(('Please select an artwork', ''), ('Beanstalk (1995-7-1.json)', '1995-7-1.json'), ('Mask wit…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<div id='vis' style='height:100%;width:6000px'></div>